In [1]:
from Genie_Algorithmic import create_model_and_diffusion

/home/gcilleru/ProphetNet/GENIE/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--------------------------------------------------------------------------
[[64892,1],0]: A high-performance Open MPI point-to-point messaging module
was unable to find any relevant network interfaces:

Module: OpenFabrics (openib)
  Host: hrist

Another transport will be used instead, although this may result in
lower performance.

NOTE: You can disable this warning by setting the MCA parameter
btl_base_warn_component_unused to 0.
--------------------------------------------------------------------------


In [2]:
import argparse
parser = argparse.ArgumentParser()

# out path
parser.add_argument('--checkpoint_path', type=str, default='', help='output path')

# load pretrain
parser.add_argument('--pretrain_model_path', type=str, default=None, help='using pretraining diffusion')

# load model
parser.add_argument('--model_arch', type=str, default='transformer', help='Core architecture of diffusion model')
parser.add_argument('--model_channels', type=int, default=768, help='Try to set it to the same size as the model hidden')
parser.add_argument('--in_channel', type=int, default=768, help='The input chanel size here must be the same as the word embedding size')
parser.add_argument('--out_channel', type=int, default=768, help='The dimension size of the output is recommended to be the same as that of word embedding for easy reasoning')
parser.add_argument('--dropout', type=float, default=0.1, help='')
parser.add_argument("--learn_sigma", default=False, action="store_true", help="Whether to learning variance")
parser.add_argument('--logits_mode', type=int, default=1, help='final logits mode of Diffusion model')
parser.add_argument('--vocab_size', type=int, default=30522, help='vocab size')
parser.add_argument('--config_name', type=str, default='bert-base-uncased', help='')
parser.add_argument('--token_emb_type', type=str, default='random', help='token embedding type')
parser.add_argument("--init_pretrained", default=False, action="store_true", help="Whether to using pretrain BERT encoder")
parser.add_argument("--fix_encoder", default=False, action="store_true",
                    help="Whether to training encoder")


# load diffusion
parser.add_argument('--diffusion_steps', type=int, default=2000, help='Diffusion model maximum T')
parser.add_argument('--use_kl', default=False, action="store_true", help="Whether to using kl loss in Diffsion loss")
parser.add_argument('--training_mode', type=str, default='e2e', help='using e2e simple loss or e2e loss or s2s loss')
parser.add_argument('--noise_schedule', type=str, default='sqrt', help='How to plan the noise change of Gaussian distribution')
parser.add_argument('--predict_xstart', default=False, action="store_true", help="Model prediction target, if True, predict xstart, if False, predict EPSILON")
parser.add_argument("--sigma_small", default=False, action="store_true", help="about learning variance")
parser.add_argument("--rescale_learned_sigmas", default=True, action="store_false", help="about learning variance")
parser.add_argument("--rescale_timesteps", default=True, action="store_false", help="about time rescale")

# sample t
parser.add_argument('--schedule_sampler', type=str, default='uniform', help='how to sample t per batch, uniform is Uniform sampling, loss-second-moment is Sampling according to loss')

# data args
parser.add_argument('--data_path', type=str, default='',help='data path')
parser.add_argument('--data_name', type=str, default='', help='data name')
# for seq2seq
parser.add_argument('--src_max_len', type=int, default=144, help='src max len')
parser.add_argument('--tgt_max_len', type=int, default=32, help='tgt max len')
parser.add_argument('--answer_max_len', type=int, default=10, help='tgt max len')
# for doc2query
parser.add_argument('--text_max_len', type=int, default=None, help='text max len')
parser.add_argument('--pas_max_len', type=int, default=None, help='pas max len')

# training args
parser.add_argument('--train_type', type=str, default='LM_Diffusion', help='LM_Diffusion or S2S_Diffusion')
parser.add_argument('--lr_anneal_steps', type=int, default=200000, help='total step')
parser.add_argument('--batch_size', type=int, default=64, help='')
parser.add_argument('--lr', type=float, default=1e-04, help='')
parser.add_argument('--warmup_steps', type=int, default=20000, help='')
parser.add_argument('--ema_rate', type=str, default='0.9999', help='ema training to stable model')
parser.add_argument('--resume_checkpoint', type=str, default=None, help='')
parser.add_argument('--eval_interval', type=int, default=2000, help='')
parser.add_argument('--log_interval', type=int, default=100, help='')
parser.add_argument('--save_interval', type=int, default=50000, help='')

parser.add_argument('--weight_decay', type=str, default=0.0, help='')
parser.add_argument('--gradient_clipping', type=float, default=-1., help='')
parser.add_argument("--use_fp16", default=False, action="store_true", help="about learning variance")
parser.add_argument('--fp16_scale_growth', type=float, default=1e-3, help='')

# seed
parser.add_argument('--seed', type=int, default=101, help='')

# muti-gpu
parser.add_argument("--local_rank", type=int, default=-1, help="For distributed training: local_rank")
parser.add_argument("--server_ip", type=str, default="", help="For distant debugging.")
parser.add_argument("--server_port", type=str, default="", help="For distant debugging.")


_StoreAction(option_strings=['--server_port'], dest='server_port', nargs=None, const=None, default='', type=<class 'str'>, choices=None, help='For distant debugging.', metavar=None)

In [17]:
# get default args from parser object
args = parser.parse_args([])
args.vocab_size = 100
args.model_arch = 'tiny_CAT' #'tiny_CAT'# 's2s_CAT'

In [18]:
model, diffusion = create_model_and_diffusion(
        args
    )

creating model, based on tiny_CAT


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['bert.encoder.layer.5.intermediate.dense.weight', 'bert.encoder.layer.2.attention.self.key.weight', 'bert.encoder.layer.3.attention.self.query.weight', 'bert.encoder.layer.11.attention.output.LayerNorm.weight', 'bert.encoder.layer.7.attention.self.value.weight', 'bert.encoder.layer.5.attention.output.dense.bias', 'bert.encoder.layer.2.attention.self.query.bias', 'cls.predictions.decoder.weight', 'bert.encoder.layer.9.attention.output.dense.weight', 'bert.encoder.layer.10.intermediate.dense.weight', 'bert.encoder.layer.2.attention.output.dense.weight', 'bert.encoder.layer.8.output.dense.bias', 'bert.encoder.layer.11.intermediate.dense.weight', 'bert.encoder.layer.2.intermediate.dense.bias', 'bert.encoder.layer.6.output.LayerNorm.weight', 'bert.encoder.layer.5.attention.self.key.weight', 'bert.encoder.layer.7.output.dense.weight', 'bert.encoder.layer.2.output.dense.weight', 'bert.encoder

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

noise_schedule:  sqrt
Diffusion Steps:  2000
betas:  [0.01464131 0.00888909 0.00706818 ... 0.35722328 0.55561113 0.999     ]
Diffusion Loss Type:  LossType.E2E_MSE   , Whether to learn sigma:  False
Diffusion predict xstart:  False
training mode is  e2e


In [19]:
model

Tiny_CAS_Diffusion_LM(
  (passage_encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem